In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['STD']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [10]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 1]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [11]:
list(zip(longforms, counts))

[('saturation transfer difference', 47),
 ('short term depression', 36),
 ('sexually transmitted disease', 25),
 ('standard diet', 14),
 ('standard deviation', 10),
 ('standard', 9),
 ('stichoposide d', 7),
 ('synaptic depression', 7),
 ('spike timing dependent', 3),
 ('standard treatment', 3),
 ('samples u c c d and', 3),
 ('sodium tungstate dihydrate', 2),
 ('short term depletion', 2),
 ('seminiferous tubular diameter', 2),
 ('seminiferous tubule diameter', 2),
 ('synthesis conditions', 2)]

In [12]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [13]:
names

{'MESH:D004032': 'Diet',
 'MESH:C000609767': 'stichoposide D',
 'MESH:D013812': 'Therapeutics',
 'CHEBI:CHEBI:63939': 'sodium tungstate dihydrate',
 'HGNC:11458': 'SULT2A1',
 'MESH:D003866': 'Depressive Disorder',
 'MESH:D009682': 'Magnetic Resonance Spectroscopy',
 'MESH:D012749': 'Sexually Transmitted Diseases'}

In [14]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-10 05:09:44] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [15]:
result = [grounding_map, names, pos_labels]

In [16]:
result

[{'samples u c c d and': 'ungrounded',
  'saturation transfer difference': 'NCIT:C116451',
  'seminiferous tubular diameter': 'ungrounded',
  'seminiferous tubule diameter': 'ungrounded',
  'sexually transmitted disease': 'MESH:D012749',
  'short term depletion': 'ungrounded',
  'short term depression': 'ungrounded',
  'sodium tungstate dihydrate': 'CHEBI:CHEBI:63939',
  'spike timing dependent': 'ungrounded',
  'standard': 'ungrounded',
  'standard deviation': 'ungrounded',
  'standard diet': 'ungrounded',
  'standard treatment': 'ungrounded',
  'stichoposide d': 'MESH:C000609767',
  'synaptic depression': 'ungrounded',
  'synthesis conditions': 'ungrounded'},
 {'NCIT:C116451': 'Chemical Exchange Saturation Transfer Magnetic Resonance Imaging',
  'MESH:D012749': 'Sexually Transmitted Diseases',
  'CHEBI:CHEBI:63939': 'sodium tungstate dihydrate',
  'MESH:C000609767': 'stichoposide D'},
 ['CHEBI:CHEBI:63939']]

In [17]:
grounding_map, names, pos_labels = [{'samples u c c d and': 'ungrounded',
  'saturation transfer difference': 'NCIT:C116451',
  'seminiferous tubular diameter': 'ungrounded',
  'seminiferous tubule diameter': 'ungrounded',
  'sexually transmitted disease': 'MESH:D012749',
  'short term depletion': 'ungrounded',
  'short term depression': 'ungrounded',
  'sodium tungstate dihydrate': 'CHEBI:CHEBI:63939',
  'spike timing dependent': 'ungrounded',
  'standard': 'ungrounded',
  'standard deviation': 'ungrounded',
  'standard diet': 'ungrounded',
  'standard treatment': 'ungrounded',
  'stichoposide d': 'MESH:C000609767',
  'synaptic depression': 'ungrounded',
  'synthesis conditions': 'ungrounded'},
 {'NCIT:C116451': 'Chemical Exchange Saturation Transfer Magnetic Resonance Imaging',
  'MESH:D012749': 'Sexually Transmitted Diseases',
  'CHEBI:CHEBI:63939': 'sodium tungstate dihydrate',
  'MESH:C000609767': 'stichoposide D'},
 ['CHEBI:CHEBI:63939']]

In [18]:
excluded_longforms = []

In [19]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [35]:
additional_entities = {'HGNC:11458': ['SULT2A1', ['STD']]}

In [36]:
unambiguous_agent_texts = {}

In [37]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [38]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [39]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [40]:
intersection1

[('HGNC:11458', 'HGNC:11458', 375)]

In [41]:
intersection2

[('MESH:D012749', 'HGNC:11458', 0),
 ('ungrounded', 'HGNC:11458', 0),
 ('NCIT:C116451', 'HGNC:11458', 0),
 ('MESH:C000609767', 'HGNC:11458', 0),
 ('CHEBI:CHEBI:63939', 'HGNC:11458', 0)]

In [42]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [43]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [44]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-10 05:19:12] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-10 05:19:13] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [45]:
classifier.stats

{'label_distribution': {'MESH:D012749': 21,
  'ungrounded': 68,
  'NCIT:C116451': 31,
  'MESH:C000609767': 4,
  'CHEBI:CHEBI:63939': 1,
  'HGNC:11458': 5},
 'f1': {'mean': 0.9, 'std': 0.2},
 'precision': {'mean': 0.9, 'std': 0.2},
 'recall': {'mean': 0.9, 'std': 0.2},
 'CHEBI:CHEBI:63939': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D012749': {'f1': {'mean': 0.949206, 'std': 0.063014},
  'pr': {'mean': 0.95, 'std': 0.1},
  'rc': {'mean': 0.96, 'std': 0.08}},
 'MESH:C000609767': {'f1': {'mean': 0.6, 'std': 0.489898},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'NCIT:C116451': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.970822, 'std': 0.02842},
  'pr': {'mean': 0.984615, 'std': 0.030769},
  'rc': {'mean': 0.957949, 'std': 0.034538}},
 'HGNC:11458': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {

In [46]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [47]:
disamb.dump(model_name, results_path)

In [48]:
print(disamb.info())

Disambiguation model for STD

Produces the disambiguations:
	Chemical Exchange Saturation Transfer Magnetic Resonance Imaging	NCIT:C116451
	SULT2A1*	HGNC:11458
	Sexually Transmitted Diseases	MESH:D012749
	sodium tungstate dihydrate*	CHEBI:CHEBI:63939
	stichoposide D	MESH:C000609767

Class level metrics:
--------------------
Grounding                                                       	Count	F1     
                                                      Ungrounded	68	0.97082
Chemical Exchange Saturation Transfer Magnetic Resonance Imaging	31	    1.0
                                   Sexually Transmitted Diseases	21	0.94921
                                                         SULT2A1*	 5	    1.0
                                                  stichoposide D	 4	    0.6
                                      sodium tungstate dihydrate*	 1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.9
	Precision:	0.9
	Recall:		0.9

* Positive labels
See Docstring for explanation



In [52]:
model_to_s3(disamb)

In [49]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [50]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:11458']

0